Notebook for Twitter Topic Modeling

In [1]:
#pip install git+https://github.com/rwalk/gsdmm.git

In [2]:
import gsdmm
import numpy as np
import pandas as pd
import pickle
import re

from gsdmm import MovieGroupProcess
from tqdm import tqdm

In [3]:
# Import dataset
tweets_df = pd.read_csv('dummy data_processed')
tweets_df.head()

,Unnamed: 0,Unnamed: 0.1,tweet_id,text,processed_text,stripped_text,text_lem,text_tokens
0,0,0,1630989944289398784,"I ""think"" I started my menstrual cycle yesterd...","i ""think"" i started my menstrual cycle yesterd...",i think i started my menstrual cycle yesterd...,i think i started my menstrual cycle yesterd...,"['i', 'think', 'i', 'started', 'my', 'menstrua..."
1,1,1,1630974067875381265,cw menstruation ////\n.\n.\n.\ni have had my p...,cw menstruation //// . . . i have had my perio...,cw menstruation i have had my period f...,cw menstruation i have had my period f...,"['cw', 'menstruation', 'i', 'have', 'had', 'my..."
2,2,2,1630965437709053952,Clubs must ensure that they enhance supportive...,clubs must ensure that they enhance supportive...,clubs must ensure that they enhance supportive...,clubs must ensure that they enhance supportive...,"['clubs', 'must', 'ensure', 'that', 'they', 'e..."
3,3,3,1630878576538013696,#EndPeriodShaming\nI used to think it was a no...,i used to think it was a normal narrative unt...,i used to think it was a normal narrative unt...,i used to think it was a normal narrative unt...,"['i', 'used', 'to', 'think', 'it', 'was', 'a',..."
4,4,4,1630863346143580161,"Let’s push for ending period stigma, period po...","let us push for ending period stigma, period p...",let us push for ending period stigma period p...,let us push for ending period stigma period p...,"['let', 'us', 'push', 'for', 'ending', 'period..."


In [4]:
# Create a single list of tweet tokens
docs = tweets_df['text_tokens'].tolist()
docs[:3]

["['i', 'think', 'i', 'started', 'my', 'menstrual', 'cycle', 'yesterday', 'but', 'due', 'to', 'the', 'ablation', 'surgery', 'working', 'i', 'am', 'not', 'bleeding', 'at', 'all', 'it', 'is', 'really', 'bizarre', 'and', 'cool', 'i', 'have', 'always', 'been', 'regular', 'so', 'i', 'can', 'track', 'that', 'it', 'is', 'the', 'right', 'time', 'but', 'the', 'period', 'part', 'of', 'it', 'is', 'just', 'not', 'there', 'at', 'all', 'one', 'still…']",
 "['cw', 'menstruation', 'i', 'have', 'had', 'my', 'period', 'for', 'over', '15', 'years', 'my', 'cycle', 'has', 'always', 'been', 'over', 'a', 'month', 'when', 'will', 'my', 'brain', 'stop', 'convincing', 'myself', 'that', 'i', 'am', 'pregnant', 'every', 'time', 'i', 'hit', 'day', '29', 'of', 'my', 'cycle']",
 "['clubs', 'must', 'ensure', 'that', 'they', 'enhance', 'supportive', 'measures', 'to', 'our', 'women', 'on', 'matters', 'menstrual', 'health', 'hygiene', 'management', 'mhm', 'including', 'period', 'tracking', 'free', 'sanitary', 'pads', 'mh